# PPMI data preparation script

2018-09-10 Hirotaka Iwaki

Before staring the analysis, Donwnload data from PPMI LONI site.

**steps**
1. Create 1-1 matching table for Event ID and DATE of each participant. 
2. Pull PD diagnosis information

In [1]:
# Setting
library(data.table)
library(dplyr)
FOLDER = c("PPMI180910")
OUTPUT = c("out180910")


Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



## Step 1 PATNO-EVENT-DATE matching table

In [ ]:
cat("Populate DATE and EVENT_ID from all the available files.\n
Skip 'Signature_Form.csv' because it contains many EVENT_ID which are not seen in other files.
Additionally, the following files will be skipped because they don't contain PATNO in the file;\n ")
FILES = dir(path = FOLDER, full.names = F, recursive = T)
TEMP=c()
for (i in 1:length(FILES)){
  if(FILES[i]=="Signature_Form.csv"){next} 
  CHECK = fread(paste(FOLDER, FILES[i], sep="/"))
  if(length(grep("^PATNO$", names(CHECK)))==0){cat(FILES[i], ", ");next}
  TEMP1 = fread(paste(FOLDER, FILES[i], sep="/"), na.strings = '', header = T, quote="\"", 
                colClasses = c("PATNO"="character")) 
  if(length(grep("^EVENT_ID$|^INFODT$", names(TEMP1)))==2){
    TEMP2 = TEMP1 %>% select(PATNO, EVENT_ID, INFODT) %>% 
      mutate(DATE = as.numeric(as.Date(paste("01", INFODT, sep="/"), format="%d/%m/%Y"))) %>% # pad 01 and cal days from 1970-01-01
      arrange(DATE) %>% 
      distinct(PATNO, EVENT_ID, .keep_all = T) %>%
      .[complete.cases(.),]%>% 
      select(PATNO, EVENT_ID, DATE) %>% mutate(FILE = FILES[i]) %>% data.frame
    if(!exists("TEMP")){TEMP=TEMP2}else{
      TEMP = bind_rows(TEMP, TEMP2)
    }
  }else{next}
}


cat("\n \n \n Check how consistent the EVENT_ID-DATEs are. 
Calculate P: the probability of a certain event of individual recorded as such a date\n")
temp = TEMP %>% arrange(DATE) %>% 
  group_by(PATNO, EVENT_ID) %>% mutate(NUM_E = n()) %>% data.frame %>%  
  group_by(PATNO, EVENT_ID, DATE) %>% mutate(NUM_E_D = n()) %>% data.frame %>% 
  mutate(P = NUM_E_D / NUM_E) %>% 
  distinct(PATNO, EVENT_ID, NUM_E_D, .keep_all = T) %>%
  arrange(PATNO, DATE) 

cat("\n\nList of files that contains only 1 event for any participant across all files. Numbers are such obs in the file.")
temp %>%  filter(NUM_E == 1) %>% with(table(FILE, NUM_E==1))
temp %>% filter(P<1) %>% nrow %>% cat("\n", ., "observations have different DATEs for the same EVENT_ID.
For these events, take the most frequent DATE")
temp2 = temp %>% arrange(PATNO, EVENT_ID, desc(P)) %>% distinct(PATNO, EVENT_ID, .keep_all = T) 
temp2 %>% filter(P == 0.5) %>% nrow %>% cat("\nFor", ., "observations, the most freqent DATE of the EVENT_ID is 0.5,
DATEs of those EVENT_IDs will be determined by the order of the filenames (because we cannot determine which is true).\n
Next, check the chronological order of EVENT_ID. 
EVENT_ID of schduled site visits and telephone visits are used to detect inconsistency.\n")

# Code to see the names of EVENT_IDs
# unique(temp$EVENT_ID) %>% .[-grep("^T", .)] %>% paste(., collapse = '","')

Visits = c("SC", "BL", sprintf("%s%02d", "V", 1:20))
TVisits= c("TSC", "TBL", sprintf("%s%02d", "T", 1:99))
temp3.1 = temp2 %>% 
  filter(EVENT_ID %in% Visits) %>% 
  mutate(V = factor(EVENT_ID, levels = Visits)) %>% 
  arrange(PATNO, V) %>% 
  group_by(PATNO) %>% 
  mutate(DATEbef = lag(DATE, default = first(DATE)),
         DATEaft = lead(DATE, default = last(DATE))) %>% 
  data.frame %>% filter(DATE < DATEbef | DATE > DATEaft ) %>% select(-"V")
temp3.2 = temp2 %>% 
  filter(EVENT_ID %in% TVisits) %>% 
  mutate(V = factor(EVENT_ID, levels = TVisits)) %>% 
  arrange(PATNO, V) %>% 
  group_by(PATNO) %>% 
  mutate(DATEbef = lag(DATE, default = first(DATE)),
         DATEaft = lead(DATE, default = last(DATE))) %>% 
  data.frame %>% filter(DATE < DATEbef | DATE > DATEaft ) %>% select(-"V")
temp3 = bind_rows(temp3.1, temp3.2) # the list of not chronologially consistent events
nrow(temp2) %>% cat("\nAmong", ., "obs, ")
nrow(temp3) %>% cat(., " will be deleted because chronological order is inconsitent")
temp4 = anti_join(temp2, temp3, by=c("PATNO", "EVENT_ID"))
cat("\nAfter exclude these obs, check chronological orders again.")
temp4 %>% 
  filter(EVENT_ID %in% Visits) %>% 
  mutate(V = factor(EVENT_ID, levels = Visits)) %>% 
  arrange(PATNO, V) %>% 
  group_by(PATNO) %>% 
  mutate(DATEdiff = DATE - lag(DATE, default = first(DATE))) %>% 
  data.frame %>% filter(DATEdiff<0) %>% select(-"V") %>% nrow %>%
  cat("\n",., ": Number of obs still problematic. 0 is expected." )
temp4 %>% 
  filter(EVENT_ID %in% TVisits) %>% 
  mutate(V = factor(EVENT_ID, levels = TVisits)) %>% 
  arrange(PATNO, V) %>% 
  group_by(PATNO) %>% 
  mutate(DATEdiff = DATE - lag(DATE, default = first(DATE))) %>% 
  data.frame %>% filter(DATEdiff<0) %>% select(-"V")%>% nrow %>%
  cat("\n",., ": Number of obs still problematic. 0 is expected." )
cat("\nSave PATNO_EVENT_DATE matching file")
EDtable = temp4 %>% select(PATNO, EVENT_ID, DATE)
write.csv(EDtable, paste(OUTPUT, "PATNO_EVENTID_DATE.csv", sep="/"), row.names=F)

Populate DATE and EVENT_ID from all the available files.

Skip 'Signature_Form.csv' because it contains many EVENT_ID which are not seen in other files.
Additionally, the following files will be skipped because they don't contain PATNO in the file;
 

## Step 2 PD diagnosis

In [ ]:
cat("Pull information from 'Patient_Status.csv'\n\n")
temp = fread(paste(FOLDER, 'Patient_Status.csv', sep='/'), 
                 colClasses = c("PATNO"="character")) %>%
  mutate(RECRUIT = ifelse(is.na(ENROLL_CAT), RECRUITMENT_CAT, ENROLL_CAT),
         IMG = IMAGING_CAT,
         SUBCAT = DESCRP_CAT) %>% 
  select(PATNO, RECRUIT, IMG, SUBCAT, ENROLL_DATE, ENROLL_STATUS)
cat("Check ENROLL_STATUS by contingency table. 
Note:If dup > 1 it indiates duplicated IDs in the file and need to be checked.\n")
temp %>% group_by(PATNO) %>% mutate(dup = n()) %>% with(table(dup, ENROLL_STATUS, useNA = "always"))
cat("\nContingency table for cohorts and image results. RECRUIT=PD has SWEDD category. GEN has image but not REG")
temp %>% with(table(RECRUIT, IMG, useNA = "always"))
cat("\nContingency table for RECRUIT cohort and Subcategories. RECRUIT=PRODROMA has HYP and RBD. RECRUIT=GEN/REG have genetics information")
temp %>% with(table(RECRUIT, SUBCAT, useNA = "always"))
STATUS = temp

In [ ]:
cat("Obatin the initial diagnosis, and the latest diagnosis from 'Primary_Diagnosis.csv'.\n\n")
dxcode=data.frame(
    CODE = c(1:18, 23, 24, 97), 
    DX = c("iPD", "Alz", "FTD_Chr17", "CBD", "DLB", "Dysto_DopaR", "ET", "Hemi_PKS", "ARPD", "MNDwPKS",
            "MSA", "DrugPKS", "NPH", "PSP", "Psychogenic", "VasPKS", "NoDisease", "SCA", "PrdrNM", "PrdrM","OtherD"))
diag1=fread(paste(FOLDER, 'Primary_Diagnosis.csv', sep='/'), 
            colClasses = c("PATNO"="character")) %>%
  filter(!(is.na(PATNO))) %>% 
  mutate(DXCODE = as.numeric(PRIMDIAG)) %>% 
  select(PATNO, EVENT_ID, DXCODE) %>% 
  inner_join(., EDtable, by = c("PATNO", "EVENT_ID")) %>%
  arrange(desc(DATE)) %>% 
  distinct(PATNO, .keep_all = T) %>%
  left_join(., dxcode, by = c("DXCODE"="CODE")) %>% 
  rename(DATE_LASTDX = DATE, DX_LAST = DX, EVENT_LAST=EVENT_ID)
diag2=fread(paste(FOLDER, 'Primary_Diagnosis.csv', sep='/'), 
            colClasses = c("PATNO"="character")) %>%
  filter(!(is.na(PATNO))) %>% 
  mutate(DXCODE = as.numeric(PRIMDIAG)) %>% 
  select(PATNO, EVENT_ID, DXCODE) %>% 
  inner_join(., EDtable, by = c("PATNO", "EVENT_ID")) %>%
  arrange(DATE) %>% 
  distinct(PATNO, .keep_all = T) %>%
  left_join(., dxcode, by = c("DXCODE"="CODE")) %>% 
  rename(DATE_INITDX = DATE, DX_INIT = DX, EVENT_INIT=EVENT_ID) 
diag_PPMI = full_join(diag1, diag2, by = "PATNO") %>% select(-starts_with("DXCODE"))
cat("Contingency table for DX_LAST x RECRUIT cohort. The file only covers primary PPMI cohorts")
left_join(STATUS, diag_PPMI, by = "PATNO") %>% mutate(DX_LAST = as.character(DX_LAST)) %>% with(table(DX_LAST, RECRUIT, useNA = "always"))

cat("Further pull the initial and the last DX from 'Prodromal_Diagnostic_Questionnaire.csv'.")
diag1=fread(paste(FOLDER, 'Prodromal_Diagnostic_Questionnaire.csv', sep='/'), 
            colClasses = c("PATNO"="character")) %>%
  filter(!(is.na(PATNO))) %>% 
  mutate(DXCODE = as.numeric(PRIMDIAG)) %>% 
  select(PATNO, EVENT_ID, DXCODE) %>% 
  inner_join(., EDtable, by = c("PATNO", "EVENT_ID")) %>%
  arrange(desc(DATE)) %>% 
  distinct(PATNO, .keep_all = T) %>%
  left_join(., dxcode, by = c("DXCODE"="CODE")) %>% 
  rename(DATE_LASTDX = DATE, DX_LAST = DX, EVENT_LAST=EVENT_ID)
diag2=fread(paste(FOLDER, 'Prodromal_Diagnostic_Questionnaire.csv', sep='/'), 
            colClasses = c("PATNO"="character")) %>%
  filter(!(is.na(PATNO))) %>% 
  mutate(DXCODE = as.numeric(PRIMDIAG)) %>% 
  select(PATNO, EVENT_ID, DXCODE) %>% 
  inner_join(., EDtable, by = c("PATNO", "EVENT_ID")) %>%
  arrange(DATE) %>% 
  distinct(PATNO, .keep_all = T) %>%
  left_join(., dxcode, by = c("DXCODE"="CODE")) %>% 
  rename(DATE_INITDX = DATE, DX_INIT = DX, EVENT_INIT=EVENT_ID) 
diag_PROD = full_join(diag1, diag2, by = "PATNO") %>% select(-starts_with("DXCODE"))
left_join(STATUS, diag_PROD, by = "PATNO") %>%  mutate(DX_LAST = as.character(DX_LAST)) %>% with(table(DX_LAST, RECRUIT, useNA = "always"))

In [ ]:
diag = bind_rows(diag_PPMI, diag_PROD) %>% mutate(DAYS_DXDIFF = DATE_LASTDX - DATE_INITDX)
cat("\nCompare the initial diagnosis and the last one. Note this contingency table includes people only had SC visit")
diag %>% mutate_at(vars(starts_with("DX")), as.character) %>% 
  with(table(DX_LAST, DX_INIT, useNA = "always"))
cat("\nCompare the initial diagnosis and the last one. Exclude people only had SC visit")
diag %>% mutate_at(vars(starts_with("DX")), as.character) %>% 
  filter(DAYS_DXDIFF>0) %>% 
  with(table(DX_LAST, DX_INIT, useNA = "always"))

In [ ]:
temp = left_join(STATUS, diag, by = "PATNO")
cat("Now all people enrolled have diagnosis. \n UP : recruitment category and inital diagnosis \n BOTTOM : the same but with latest diagnosis. \n 13 recruited as PD were diagnosed differently")
temp %>% filter(ENROLL_STATUS == "Enrolled") %>% mutate(DX_INIT = as.character(DX_INIT)) %>% with(table(DX_INIT, RECRUIT, useNA = "always"))
temp %>% filter(ENROLL_STATUS == "Enrolled") %>% mutate(DX_LAST = as.character(DX_LAST)) %>% with(table(DX_LAST, RECRUIT, useNA = "always"))

In [ ]:
cat("Create New Status/Diagnosis/Image Indicator")
temp1= temp %>% filter(!is.na(DX_INIT)) %>%
  mutate(DIAG = paste(
  ifelse(ENROLL_STATUS=="Enrolled", "In_", "Out"),
  case_when(
    is.na(DX_LAST) ~ "XXX",
    DX_LAST == "iPD" ~ "iPD",
    DX_LAST == "NoDisease" ~ "CTR",
    DX_LAST == "PrdrM" | DX_LAST == "PrdrNM" ~ "PRD",
    DX_LAST == "OtherD" ~ "DFR",
    DX_LAST == "ET" ~ "EST",
    DX_LAST == "DrugPKS" ~ "DRG",
    DX_LAST == "Psychogenic" ~ "PSY",
    TRUE ~ as.character(DX_LAST)),
  case_when(
    IMG == "SWEDD" ~ "SWEDD",
    IMG == "no image" ~ "NoImg",
    TRUE ~ "YsImg"),
  ifelse(SUBCAT=="" & I(RECRUIT %in% c("PD", "HC")) , "PPMI", SUBCAT),
  sep = "_"
))
cat("STATUS of PPMI/HC cohort")
temp1 %>% .[grep("PPMI", .$DIAG), ] %>% 
  with(table(DIAG, RECRUIT, useNA = "always"))
temp1 %>% .[-grep("PPMI", .$DIAG), ] %>% 
  with(table(DIAG, RECRUIT, useNA = "always"))

In [ ]:
fread(paste(FOLDER, 'Patient_Status.csv', sep='/'), 
                 colClasses = c("PATNO"="character")) %>%
  mutate(RECRUIT = RECRUITMENT_CAT,
         ENRLCAT = ENROLL_CAT,
         IMG = IMAGING_CAT,
         SUBCAT = DESCRP_CAT) %>% 
  select(PATNO, RECRUIT, IMG, SUBCAT, ENROLL_DATE, ENROLL_STATUS, ENRLCAT) %>% 
  with(table(RECRUIT, ENRLCAT, useNA = "always"))
